## __Step 1:__ Library alignment and droplet count estimation with R

The goal of Step 1 is to set up the containerized environment for running the DropEst pipeline, running the the process of barcode demultiplexing, read alignment, and droplet count estimation. The inputs to this step are the raw sequencing files and the outputs to this step are an estimated droplet read count matrix and its corresponding quality control reports.

---
### __1.0__ Before You Begin

This protocol requires three types of input files, which will have to be in an accessible directory:
    
    - A primary assembly .fa file corresponding to the reference genome for read alignment. Here we use release 85 of the Ensembl human reference genome.
        - i.e.: Homo_sapiens.GRCh38.dna_sm.primary_assembly.fa
    - A gene annotation file corresponding to the same reference genome and release version
        - i.e.: Homo_sapiens.GRCh38.85.annotated.gtf
    - Two .fastq files, corresponding to the paired R1 and R2 sequencing reads of the target sequenced single-cell RNA-seq library
        - i.e.: 3907-AS-1-CCGCGGTT-AGCGCTAG_S343_sub_R1_001.fastq and 3907-AS-1-CCGCGGTT-AGCGCTAG_S343_sub_R2_001.fastq

---
### __1.1__ Singularity Installation

To use the provided containerized environment, we need to install the required golang libraries and Singularity itself. For Ubuntu, a Debian-based linux distribution, we first update our software libraries through apt-get

In [ ]:
!sudo apt-get update && sudo apt-get install -y \
    build-essential \
    libssl-dev \
    uuid-dev \
    libgpgme11-dev \
    squashfs-tools \
    libseccomp-dev \
    pkg-config

Next install golang, by obtaining the installation files, extracting them, and updating paths for usage. In this case version 1.15.6 is used. Further instructions can be found at: https://golang.org/doc/install

In [ ]:
#Download to current directory
!wget https://dl.google.com/go/go1.15.6.linux-amd64.tar.gz

In [ ]:
#Extract files to local directory
!sudo tar -C /usr/local -xzf go1.15.6.linux-amd64.tar.gz

In [ ]:
#Update path
!export PATH=$PATH:/usr/local/go/bin

In [ ]:
#Double check installation by checking go version
!go version

Now we can install Singularity from a release, in this case version 3.7.0 is used. Further instructions can be found at: https://sylabs.io/guides/3.0/user-guide/installation.html#download-and-install-singularity-from-a-release

In [ ]:
#Download and extract the necessary Singularity libraries
!export VERSION=3.7.0 && # adjust this as necessary \
    mkdir -p $GOPATH/src/github.com/sylabs && \
    cd $GOPATH/src/github.com/sylabs && \
    wget https://github.com/sylabs/singularity/releases/download/v${VERSION}/singularity-${VERSION}.tar.gz && \
    tar -xzf singularity-${VERSION}.tar.gz && \
    cd ./singularity && \
    ./mconfig

In [ ]:
#Navigate to the installation directory and compile
!cd /home/$USER/go/src/github.com/sylabs/singularity/builddir && \
    make && \
    sudo make install

Finally, pull the Singularity image from dockerhub, this may take a while depending on download speeds. This container comes preconfigured with DropEst, STAR, and scRNABatchQC, along with their supporting libraries.

In [ ]:
!singularity pull docker://ramiremars/star_dropest:star_protocols_pipeline

---
### __1.2__ Runing DropTag

DropTag is first run to demultiplex the .fastq read files. This is the first step of the DropEst pipeline, and further documentation can be found at: https://github.com/hms-dbmi/dropEst

In [ ]:
!singularity exec -e star_dropest_star_protocols_pipeline.sif droptag -c /usr/share/dropEst/configs/indrop_v1_2.xml \
    /path_to/3907-AS-1-CCGCGGTT-AGCGCTAG_S343_sub_R1_001.fastq /path_to/3907-AS-1-CCGCGGTT-AGCGCTAG_S343_sub_R2_001.fastq

---
### __1.3__ Generating the STAR index

The STAR aligner used for read alignment requires an index file to be generated, we generate it with the following command, which creates a directory to operate within. Further documentation can be found at: https://github.com/alexdobin/STAR

In [ ]:
!mkdir STAR_index && singularity exec -e star_dropest_star_protocols_pipeline.sif STAR --runThreadN 16 --runMode genomeGenerate \
    --genomeDir STAR_index --genomeFastaFiles /path_to/Homo_sapiens.GRCh38.dna_sm.primary_assembly.fa --sjdbGTFfile /path_to/Homo_sapiens.GRCh38.85.annotated.gtf --sjdbOverhang 99


---
### __1.4__ Read alignment with the STAR aligner

STAR aligner is a splice-aware aligner which we use to align sequenced reads to the reference genome. In this example, there is only one tagged fastq file, this number will increase with read depth, the --readFilesIn parameter will need to be adjusted accordingly to include all such tagged fastq files.

In [ ]:
!singularity exec -e star_dropest_star_protocols_pipeline.sif STAR --genomeDir /path_to/STAR_index \
    --readFilesIn /path_to/3907-AS-1-CCGCGGTT-AGCGCTAG_S343_sub_R2_001.fastq.tagged.1.fastq.gz  --outSAMmultNmax 1 --runThreadN 6 --readNameSeparator space --outSAMunmapped Within --outSAMtype BAM SortedByCoordinate --outFileNamePrefix 3907-AS-1-CCGCGGTT-AGCGCTAG_S343_sub  --readFilesCommand gunzip -c


---
### __1.5__ Droplet count estimation with DropEst

STAR aligner is a splice-aware aligner which we use to align sequenced reads to the reference genome. This step may take several hours depending on the read depth of the library.

In [ ]:
!singularity exec -e star_dropest_star_protocols_pipeline.sif dropest -m -V -b -F -o sample_name -g /path_to/Homo_sapiens.GRCh38.85.annotated.gtf -L eiEIBA -c /usr/share/dropEst/configs/indrop_v1_2.xml /path_to/3907-AS-1-CCGCGGTT-AGCGCTAG_S343_subAligned.sortedByCoord.out.bam

---
### __1.6__ RDS conversion to sparse droplet count matrix

The output of DropEst used in this step contains the droplet matrix that will go into downstream scRNA-seq analyses and further droplet matrix quality control. Our script, RDS_to_sparsematrix.r generates a .mtx along with its corresponding column and row names so that analysis can be performed in non-R environments.

In [ ]:
!singularity exec -e star_dropest_star_protocols_pipeline.sif  R --vanilla --slave -f /R_scripts/RDS_to_sparesematrix.r --args /path_to/dropEst.rds

---
### __1.7__ scRNABatchQC report generation

Finally, initial droplet estimation QC metrics can be generated from the output R file from step 1.5. Further documentation can be found at: https://github.com/liuqivandy/scRNABatchQC

In [ ]:
!singularity exec -e star_dropest_star_protocols_pipeline.sif R --vanilla --slave -f /R_scripts/scRNABatchQC.r --args hsapiens /path_to/sample.csv